In [1]:
from pyspark.sql import functions as f

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1574086499174_0002,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
from pipelines.flightaware.flightaware import FlightAware

In [3]:
out = FlightAware(environment='prod')

Spark Session: livy-session-1
3.6.8 (default, Aug  2 2019, 17:42:44) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-28)] {'defaultParallelism': 20, 'ExecutorMemoryStatus.size': 3}

In [4]:
out.save_to_object_map_path = False

In [6]:
out.flightaware()

Dataframe name: arrival
root
 |-- aat: timestamp (nullable = true)
 |-- atcident: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- edt: timestamp (nullable = true)
 |-- eta: timestamp (nullable = true)
 |-- ete: string (nullable = true)
 |-- facility_hash: string (nullable = true)
 |-- facility_name: string (nullable = true)
 |-- id: string (nullable = true)
 |-- ident: string (nullable = true)
 |-- orig: string (nullable = true)
 |-- pitr: timestamp (nullable = true)
 |-- reg: string (nullable = true)
 |-- synthetic: string (nullable = true)
 |-- timeType: string (nullable = true)
 |-- type: string (nullable = true)
 |-- _input_file_name: string (nullable = false)
 |-- _record_ingest_date: timestamp (nullable = true)
 |-- record_ingest_time: string (nullable = true)
 |-- record_ingest_date: date (nullable = true)
 |-- record_ingest_hour: integer (nullable = true)
 |-- id_timestamp: timestamp (nullable = true)
 |-- id_date: date (nullable = true)
 |-- id_hour: string (

In [7]:
out.load('flightplan')


Try loading "flightplan" from "s3://sita-coe-ds-prod-v1/mart/flightaware/flightplan"...
Success! 10.9 seconds
DataFrame[aircrafttype: string, alt: string, atcident: string, dest: string, edt: timestamp, eta: timestamp, ete: string, facility_hash: string, facility_name: string, fdt: timestamp, gs: string, hexid: string, id: string, ident: string, lat: decimal(11,7), lon: decimal(11,7), orig: string, pitr: timestamp, reg: string, route: string, speed: string, status: string, trueCancel: string, type: string, _input_file_name: string, _record_ingest_date: timestamp, record_ingest_time: string, record_ingest_hour: int, id_timestamp: timestamp, id_date: date, id_hour: string, record_ingest_date: date]

In [ ]:
files = out.dataframes.flightplan.select('record_ingest_date', 'record_ingest_time', '_record_ingest_date', '_input_file_name').distinct()
files.cache()

In [ ]:
from utils.date import date_range
dt_range = date_range('2016-10-01', '2019-11-15', in_format='%Y-%m-%d', out_format='%Y-%m-%d')
dt_range = [(dt,) for dt in dt_range]
dt = spark.createDataFrame(dt_range, ['record_ingest_date']).withColumn('record_ingest_date', f.to_date('record_ingest_date', 'yyyy-MM-dd'))


In [ ]:
files_summary = files.groupBy('record_ingest_date', 'record_ingest_time').agg(f.count('_input_file_name').alias('count_files'))
files_summary.cache()

In [ ]:
minutes = files_summary.select('record_ingest_time').distinct()
df = dt.crossJoin(minutes).join(files_summary, on=['record_ingest_date', 'record_ingest_time'], how='left')
df.cache()

In [ ]:
df.filter((f.col('count_files').isNull()) | (f.col('count_files')>f.lit(1))).orderBy('record_ingest_date', 'record_ingest_time').repartition(1).write.csv('s3://sita-coe-ds-prod-v1/files_analysis/missing_or_duplicate_minutes/', header=True)

In [17]:
%%sh
cat /etc/bash.bashrc

# System-wide .bashrc file for interactive bash(1) shells.

# To enable the settings / commands in this file for login shells as well,
# this file has to be sourced in /etc/profile.

# If not running interactively, don't do anything
[ -z "$PS1" ] && return

# check the window size after each command and, if necessary,
# update the values of LINES and COLUMNS.
shopt -s checkwinsize

# set variable identifying the chroot you work in (used in the prompt below)
if [ -z "${debian_chroot:-}" ] && [ -r /etc/debian_chroot ]; then
    debian_chroot=$(cat /etc/debian_chroot)
fi

# set a fancy prompt (non-color, overwrite the one in /etc/profile)
# but only if not SUDOing and have SUDO_PS1 set; then assume smart user.
if ! [ -n "${SUDO_USER}" -a -n "${SUDO_PS1}" ]; then
  PS1='${debian_chroot:+($debian_chroot)}\u@\h:\w\$ '
fi

# Commented out, don't overwrite xterm -T "title" -n "icontitle" by default.
# If this is an xterm set the title to user@host:dir
#case "$TERM" in
#xterm*|rxvt*)
#    PROMP